In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io 

In [2]:
nn_p = 5
nn_w = 5
nn_tw = 1
freq = 20
m='mof'
freq = 20

df = get_data(m)
df = df[(df['nnw']==str(nn_w))&(df['nntw']==str(nn_tw))&(df['nnp']==str(nn_p))|(df['isnn']==False)] 

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



# Plots

In [3]:
df_pivot = get_heatmap_data(m,df)#[labels_order]
df_pivot_rank = df_pivot.rank(axis=1)
df_pivot = df_pivot.reset_index()
df_pivot.experiment = df_pivot.experiment.str.slice(3)
df_pivot.set_index(['experiment','function','freq'], inplace=True)

In [4]:
df_pivot_rank = df_pivot_rank.query(f'freq=={freq}')
df_pivot = df_pivot.query(f'freq=={freq}')

In [5]:
df_pivot.style.background_gradient(cmap=plt.cm.Greens, axis=1)

invalid value encountered in less


In [6]:
df_pivot_rank.style.background_gradient(cmap=plt.cm.Greens, axis=1)

In [7]:
summary = df_pivot.groupby(['experiment', 'function']).mean()
summary.round(2).style.background_gradient(cmap=plt.cm.Greens, axis=1)

In [8]:
out = summary.round(2).style
for i in range(len(labels_order)//2): 
    out = out.highlight_min(axis=1, subset=labels_order[i*2:(i+1)*2])

out


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


KeyError: "None of [Index(['noNN_CwN', 'NN_CwN'], dtype='object', name='method')] are in the [columns]"

# Interactive

In [9]:
from ipywidgets import interact
def show_table(dm):
    out = summary[[f'noNN_{dm}', f'NN_{dm}']].round(2)
    return out.style.background_gradient(cmap=plt.cm.Blues_r, axis=1)
    
interact(show_table, dm=['RI', 'HMu','No','Rst'])

interactive(children=(Dropdown(description='dm', options=('RI', 'HMu', 'No', 'Rst'), value='RI'), Output()), _…

<function __main__.show_table(dm)>